#### Importing Libraries

In [2]:
!pip install langdetect
!pip install webdriver_manager

  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=ed3d7cd29e2da3a7882b8a117c4ef4f6e2da550b8e91d6fa2ea197c7d5c2105c
  Stored in directory: /home/gnmarten/.cache/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests
from langdetect import detect
import re
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys

In [6]:
#imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

In [10]:
#automatically install web driver using the webdriver_manager library
#presumes Chrome is on your system
driver = webdriver.Chrome(ChromeDriverManager().install())


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - Driver [C:\Users\gnmarten\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache


#### Retrieving Top 100 Book Titles and Links to Webpage for Book

In [4]:
## Creating array from Times Top 100 Books of All Time webpage on Goodreads website

TimesTop100  = requests.get("https://www.goodreads.com/list/show/187015.Neurodiversity_related").content

## Grabbing all tags in webpage of 'a' type and class 'bookTitle'
soup = BeautifulSoup(TimesTop100,"lxml")
cont = soup.select("a.bookTitle")

## Iterating through and creating list for all titles (bookT) and links (bookLink)
bookT = [x.text.strip() for x in cont]
bookLink = ['https://www.goodreads.com'+ x.get('href') for x in cont]

## Combining list
con = np.column_stack((bookT, bookLink))
Top100Books = pd.DataFrame(con, columns = ['Book Title', 'Book Link'])

#### Retrieving Book Review Page Links and Unqiue ID for Book in Goodreads Database

In [9]:
# Retrieving Page Link for the reviews of a certain book
# requests.get has tendency to crash so must be run with a while loop until requests successfully works - load time (~11.5 min)

ReviewPageLink = []
for i in bookLink:
    cont = None
    while cont == None:
            r = requests.get(i).content
            soup = BeautifulSoup(r,"lxml")
            cont = soup.find("a", attrs = {'class' : 'Button Button--transparent Button--small'})

    ReviewPageLink += ['https://www.goodreads.com' + cont['href']]

# Finding the Unique ID set by Goodreads in each books main page href
BookID = [re.search('\d+', i)[0] for i in Top100Books['Book Link']]

# adding information to dataframe
Top100Books['Review Page Link'] = ReviewPageLink
Top100Books['Unique ID'] = BookID

# saving dataframe to csv file
Top100Books.to_csv('Top100BooksData.csv', index=False)

In [6]:
## Defining function to continuing spamming requests until html can be processed
## when looking to iterate through multiple links while scraping

# Code currently returns a search - however - might be more important to return just the soup 
# Therefore, it can be used for mulitple searches if required
# The caveat is that by the definition of the function it must find some object that does exist on the page regardless
# in order to work 

def spamRequestsFind(link, tag, attr, attr_id ):
    status = None
    while status == None:
        r = requests.get(link).content
        soup = BeautifulSoup(r, "lxml")
        status = type(soup.find(tag, attrs = {attr : attr_id}))

    return(soup.find(tag, attrs = {attr : attr_id}))

    

In [2]:
Top100Books = pd.read_csv('Top100BooksData.csv')
Top100Books.head()

,Book Title,Book Link,Review Page Link,Unique ID
0,Bewilderment,https://www.goodreads.com/book/show/56404444-b...,https://www.goodreads.com/book/show/56404444/r...,56404444


In [3]:
## MultBookReviews grabs all defined information for all book reviews loaded on the page in browser

def MultBookReviews(page_source):

    ## starting by grabbing one persons review and information
    
    soup = BeautifulSoup(page_source, 'lxml')
    ReviewCards = soup.find_all('article', attrs = {'class' : 'ReviewCard'})

    ## Book Title and Author
    title = soup.find('h1', attrs = {'class' : 'Text H1Title'}).text
    author = soup.find('h3', attrs = {'class' : 'Text Text__title3 Text__regular'}).text

    ## List of all user account hrefs for account page
    cont = soup.select('div.ReviewerProfile__name')
    hrefsUsers = [x.find('a')['href'] for x in cont]

    ## Text data of user review
    contReview = soup.select("section.ReviewText")
    Reviews = [x.text.strip() for x in contReview]

    ## grabbing individual user rating for review 
    contRatingCont = soup.select("div.ShelfStatus")
    userRatings = [x.find('span')['aria-label'] if (x.findChildren('span', recursive=False) == []) == False else 'No Rating' for x in contRatingCont]

    ## Date the review was written by user
    dateCont = soup.select('section.ReviewCard__row')
    datesOfReviews = [x.find('span', attrs = {'class': 'Text Text__body3'}).text for x in dateCont]

    ## Amount of likes and comments for review
    commentLikeCont = soup.select('footer.SocialFooter')
    likes = ['0' if x.find('div', attrs={'class': 'SocialFooter__statsContainer'}) == None else x.find('span', attrs={'class': 'Button__labelItem'}).text  for x in commentLikeCont]
    comments = ['0' if x.find('div', attrs={'class': 'Button__container'}).next_sibling == None else x.find('div', attrs={'class': 'Button__container'}).next_sibling.text for x in commentLikeCont]

    ### Creating DataFrame of all the review data

    reviewData = pd.DataFrame({ 'User Href' : hrefsUsers,
                                'Title' : title,
                                'Rating' : userRatings,
                                'Date' : datesOfReviews,
                                'Likes' : likes,
                                'Comments' : comments,
                                'Review' : Reviews})
    
    return(reviewData)

#### Creating Function to Automate clicking "show more reviews" on each book webpage

In [8]:
## getManyReviews uses selenium to load more reviews on a book review webpage
## Each iteration clicks the "show more reviews" button (loads 30 more reviews per click) up to a cap of 36 clicks
## 36 was determined as the cap before the memory of a browser page would be expended resulting in a crash

def getManyReviews(url):
    clicks = 0

    # Initilaizing driver and webpage and allowing time for reviews to load
    #driver = webdriver.Edge()
    driver.get(url)
    time.sleep(3)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    nreviews = int(re.sub('\D', '', soup.find('span', attrs = {'class' : 'Text Text__body3 Text__subdued'}).text))
    cap = 36
    iters = np.round(nreviews/30)-1

    if iters < cap:
        while clicks < iters:

            # scrolling down page to ensure click will work on "show more results" button
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Clicking "show more results" button
            SMResults = driver.find_element(By.XPATH, "//div[@class = 'Divider Divider--contents Divider--largeMargin']/div[@class = 'Button__container']/button")
            driver.execute_script("arguments[0].click();", SMResults)
            time.sleep(1)

            clicks += 1
    else:
        while clicks < cap:

            # scrolling down page to ensure click will work on "show more results" button
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Clicking "show more results" button
            SMResults = driver.find_element(By.XPATH, "//div[@class = 'Divider Divider--contents Divider--largeMargin']/div[@class = 'Button__container']/button")
            driver.execute_script("arguments[0].click();", SMResults)
            time.sleep(1)

            clicks += 1

    # grabbing reference for final state of page after n number of "show more results" button clicks
    page_source = driver.page_source

    reviews = MultBookReviews(page_source)

    driver.quit()

    return(reviews)

#### Iterating Through Batches of Books to Grab Reviews (5 books, ~1000 reviews per book, ~15 min to run)

In [11]:
# Processing in smaller batches to double check loading into csv was successful
for i in range(0, 5):
    bookdata = getManyReviews(Top100Books['Review Page Link'][i])
    bookdata.to_csv('Book{num}.csv'.format(num=i+1), index=False)
#will exit on valuerror 1 is not in range because there is only 1 book, but the CSV will be saved a Book1.csv regardless

KeyError: 1